In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 1
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 1: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 250
test_batch = 250
lr = 0.1
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b0/to_pggan/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '__'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=100, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        losses.update(loss.data.tolist(), inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, ac=arc.avg))
    return (losses.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  
                   source_loss, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    scheduler.step()
    
    if (epoch+1)%900 == 0:
        teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.100000
Train | 16/16 | Loss:1.2608 | MainLoss:1.2401 | Alpha:0.0283 | SPLoss:0.7336 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.7650 | MainLoss:0.7650 | SPLoss:1.1205 | CLSLoss:0.0000 | AUROC:0.5235
Test | 30/16 | Loss:0.2728 | MainLoss:0.2728 | SPLoss:1.1205 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [2 | 1000] LR: 0.103000
Train | 16/16 | Loss:0.7696 | MainLoss:0.7337 | Alpha:0.0295 | SPLoss:1.2158 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.7217 | MainLoss:0.7217 | SPLoss:1.2663 | CLSLoss:0.0000 | AUROC:0.5346
Test | 30/16 | Loss:0.2847 | MainLoss:0.2847 | SPLoss:1.2663 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [3 | 1000] LR: 0.106000
Train | 16/16 | Loss:0.7449 | MainLoss:0.7057 | Alpha:0.0289 | SPLoss:1.3606 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6996 | MainLoss:0.6996 | SPLoss:1.4854 | CLSLoss:0.0000 | AUROC:0.5413
Test | 30/16 | Loss:0.3282 | MainLoss:0.3282 | SPLoss:1.4854 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [4 | 100

Train | 16/16 | Loss:0.7157 | MainLoss:0.6787 | Alpha:0.0293 | SPLoss:1.2672 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6798 | MainLoss:0.6798 | SPLoss:1.3309 | CLSLoss:0.0000 | AUROC:0.6089
Test | 30/16 | Loss:0.3834 | MainLoss:0.3834 | SPLoss:1.3309 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [27 | 1000] LR: 0.178000
Train | 16/16 | Loss:0.7168 | MainLoss:0.6795 | Alpha:0.0293 | SPLoss:1.2719 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6842 | MainLoss:0.6842 | SPLoss:1.2400 | CLSLoss:0.0000 | AUROC:0.6114
Test | 30/16 | Loss:0.3026 | MainLoss:0.3026 | SPLoss:1.2400 | CLSLoss:0.0000 | AUROC:0.9996

Epoch: [28 | 1000] LR: 0.181000
Train | 16/16 | Loss:0.7143 | MainLoss:0.6777 | Alpha:0.0291 | SPLoss:1.2562 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6786 | MainLoss:0.6786 | SPLoss:1.2614 | CLSLoss:0.0000 | AUROC:0.6122
Test | 30/16 | Loss:0.3348 | MainLoss:0.3348 | SPLoss:1.2614 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [29 | 1000] LR: 0.184000
Train | 16/16

Train | 16/16 | Loss:0.7154 | MainLoss:0.6730 | Alpha:0.0292 | SPLoss:1.4454 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6620 | MainLoss:0.6620 | SPLoss:1.4079 | CLSLoss:0.0000 | AUROC:0.6840
Test | 30/16 | Loss:0.2936 | MainLoss:0.2936 | SPLoss:1.4079 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [52 | 1000] LR: 0.253000
Train | 16/16 | Loss:0.7161 | MainLoss:0.6726 | Alpha:0.0301 | SPLoss:1.4450 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6534 | MainLoss:0.6534 | SPLoss:1.5188 | CLSLoss:0.0000 | AUROC:0.6871
Test | 30/16 | Loss:0.3734 | MainLoss:0.3734 | SPLoss:1.5188 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [53 | 1000] LR: 0.256000
Train | 16/16 | Loss:0.7237 | MainLoss:0.6805 | Alpha:0.0286 | SPLoss:1.5058 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.6611 | MainLoss:0.6611 | SPLoss:1.5129 | CLSLoss:0.0000 | AUROC:0.6963
Test | 30/16 | Loss:0.2973 | MainLoss:0.2973 | SPLoss:1.5129 | CLSLoss:0.0000 | AUROC:0.9997

Epoch: [54 | 1000] LR: 0.259000
Train | 16/16

Train | 16/16 | Loss:0.6193 | MainLoss:0.4997 | Alpha:0.0288 | SPLoss:4.1562 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.4032 | MainLoss:0.4032 | SPLoss:4.0521 | CLSLoss:0.0000 | AUROC:0.9914
Test | 30/16 | Loss:0.3110 | MainLoss:0.3110 | SPLoss:4.0521 | CLSLoss:0.0000 | AUROC:0.9952

Epoch: [77 | 1000] LR: 0.328000
Train | 16/16 | Loss:0.6188 | MainLoss:0.4984 | Alpha:0.0293 | SPLoss:4.1076 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.3417 | MainLoss:0.3417 | SPLoss:4.0370 | CLSLoss:0.0000 | AUROC:0.9919
Test | 30/16 | Loss:0.3669 | MainLoss:0.3669 | SPLoss:4.0370 | CLSLoss:0.0000 | AUROC:0.9953

Epoch: [78 | 1000] LR: 0.331000
Train | 16/16 | Loss:0.6091 | MainLoss:0.4907 | Alpha:0.0287 | SPLoss:4.1177 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2568 | MainLoss:0.2568 | SPLoss:3.9817 | CLSLoss:0.0000 | AUROC:0.9916
Test | 30/16 | Loss:0.4874 | MainLoss:0.4874 | SPLoss:3.9817 | CLSLoss:0.0000 | AUROC:0.9956

Epoch: [79 | 1000] LR: 0.334000
Train | 16/16

Train | 16/16 | Loss:0.9413 | MainLoss:0.4964 | Alpha:0.0296 | SPLoss:15.4738 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2909 | MainLoss:0.2909 | SPLoss:33.8702 | CLSLoss:0.0000 | AUROC:0.9925
Test | 30/16 | Loss:0.4603 | MainLoss:0.4603 | SPLoss:33.8702 | CLSLoss:0.0000 | AUROC:0.9925

Epoch: [102 | 1000] LR: 0.400000
Train | 16/16 | Loss:1.3361 | MainLoss:0.5051 | Alpha:0.0288 | SPLoss:28.9600 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2930 | MainLoss:0.2930 | SPLoss:23.6892 | CLSLoss:0.0000 | AUROC:0.9919
Test | 30/16 | Loss:0.4408 | MainLoss:0.4408 | SPLoss:23.6892 | CLSLoss:0.0000 | AUROC:0.9944

Epoch: [103 | 1000] LR: 0.399999
Train | 16/16 | Loss:1.0850 | MainLoss:0.5002 | Alpha:0.0288 | SPLoss:20.2959 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2245 | MainLoss:0.2245 | SPLoss:17.1282 | CLSLoss:0.0000 | AUROC:0.9924
Test | 30/16 | Loss:0.5910 | MainLoss:0.5910 | SPLoss:17.1282 | CLSLoss:0.0000 | AUROC:0.9910

Epoch: [104 | 1000] LR: 0.399996
T

...

Train | 16/16 | Loss:0.5450 | MainLoss:0.4590 | Alpha:0.0296 | SPLoss:2.9039 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2295 | MainLoss:0.2295 | SPLoss:2.9020 | CLSLoss:0.0000 | AUROC:0.9953
Test | 30/16 | Loss:0.5314 | MainLoss:0.5314 | SPLoss:2.9020 | CLSLoss:0.0000 | AUROC:0.9890

Epoch: [402 | 1000] LR: 0.003176
Train | 16/16 | Loss:0.5453 | MainLoss:0.4614 | Alpha:0.0289 | SPLoss:2.9027 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2278 | MainLoss:0.2278 | SPLoss:2.9026 | CLSLoss:0.0000 | AUROC:0.9954
Test | 30/16 | Loss:0.5350 | MainLoss:0.5350 | SPLoss:2.9026 | CLSLoss:0.0000 | AUROC:0.9892

Epoch: [403 | 1000] LR: 0.003170
Train | 16/16 | Loss:0.5327 | MainLoss:0.4489 | Alpha:0.0289 | SPLoss:2.8994 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2285 | MainLoss:0.2285 | SPLoss:2.8974 | CLSLoss:0.0000 | AUROC:0.9955
Test | 30/16 | Loss:0.5304 | MainLoss:0.5304 | SPLoss:2.8974 | CLSLoss:0.0000 | AUROC:0.9892

Epoch: [404 | 1000] LR: 0.003165
Train | 16

Train | 16/16 | Loss:0.5425 | MainLoss:0.4599 | Alpha:0.0287 | SPLoss:2.8789 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2256 | MainLoss:0.2256 | SPLoss:2.8777 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5341 | MainLoss:0.5341 | SPLoss:2.8777 | CLSLoss:0.0000 | AUROC:0.9896

Epoch: [427 | 1000] LR: 0.003045
Train | 16/16 | Loss:0.5361 | MainLoss:0.4529 | Alpha:0.0289 | SPLoss:2.8787 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2244 | MainLoss:0.2244 | SPLoss:2.8778 | CLSLoss:0.0000 | AUROC:0.9955
Test | 30/16 | Loss:0.5365 | MainLoss:0.5365 | SPLoss:2.8778 | CLSLoss:0.0000 | AUROC:0.9896

Epoch: [428 | 1000] LR: 0.003040
Train | 16/16 | Loss:0.5431 | MainLoss:0.4632 | Alpha:0.0278 | SPLoss:2.8764 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2259 | MainLoss:0.2259 | SPLoss:2.8749 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5331 | MainLoss:0.5331 | SPLoss:2.8749 | CLSLoss:0.0000 | AUROC:0.9895

Epoch: [429 | 1000] LR: 0.003034
Train | 16

Train | 16/16 | Loss:0.5348 | MainLoss:0.4516 | Alpha:0.0292 | SPLoss:2.8562 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2225 | MainLoss:0.2225 | SPLoss:2.8564 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5386 | MainLoss:0.5386 | SPLoss:2.8564 | CLSLoss:0.0000 | AUROC:0.9894

Epoch: [452 | 1000] LR: 0.002908
Train | 16/16 | Loss:0.5388 | MainLoss:0.4560 | Alpha:0.0290 | SPLoss:2.8574 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2223 | MainLoss:0.2223 | SPLoss:2.8565 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5391 | MainLoss:0.5391 | SPLoss:2.8565 | CLSLoss:0.0000 | AUROC:0.9894

Epoch: [453 | 1000] LR: 0.002902
Train | 16/16 | Loss:0.5320 | MainLoss:0.4492 | Alpha:0.0290 | SPLoss:2.8548 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2217 | MainLoss:0.2217 | SPLoss:2.8572 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5404 | MainLoss:0.5404 | SPLoss:2.8572 | CLSLoss:0.0000 | AUROC:0.9895

Epoch: [454 | 1000] LR: 0.002897
Train | 16

Train | 16/16 | Loss:0.5361 | MainLoss:0.4521 | Alpha:0.0296 | SPLoss:2.8414 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2223 | MainLoss:0.2223 | SPLoss:2.8408 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5398 | MainLoss:0.5398 | SPLoss:2.8408 | CLSLoss:0.0000 | AUROC:0.9904

Epoch: [477 | 1000] LR: 0.002765
Train | 16/16 | Loss:0.5314 | MainLoss:0.4485 | Alpha:0.0292 | SPLoss:2.8409 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2223 | MainLoss:0.2223 | SPLoss:2.8387 | CLSLoss:0.0000 | AUROC:0.9955
Test | 30/16 | Loss:0.5387 | MainLoss:0.5387 | SPLoss:2.8387 | CLSLoss:0.0000 | AUROC:0.9901

Epoch: [478 | 1000] LR: 0.002760
Train | 16/16 | Loss:0.5350 | MainLoss:0.4533 | Alpha:0.0288 | SPLoss:2.8369 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2228 | MainLoss:0.2228 | SPLoss:2.8360 | CLSLoss:0.0000 | AUROC:0.9955
Test | 30/16 | Loss:0.5371 | MainLoss:0.5371 | SPLoss:2.8360 | CLSLoss:0.0000 | AUROC:0.9902

Epoch: [479 | 1000] LR: 0.002754
Train | 16

Train | 16/16 | Loss:0.5305 | MainLoss:0.4509 | Alpha:0.0282 | SPLoss:2.8277 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2261 | MainLoss:0.2261 | SPLoss:2.8262 | CLSLoss:0.0000 | AUROC:0.9955
Test | 30/16 | Loss:0.5290 | MainLoss:0.5290 | SPLoss:2.8262 | CLSLoss:0.0000 | AUROC:0.9902

Epoch: [502 | 1000] LR: 0.002618
Train | 16/16 | Loss:0.5341 | MainLoss:0.4545 | Alpha:0.0282 | SPLoss:2.8266 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2251 | MainLoss:0.2251 | SPLoss:2.8269 | CLSLoss:0.0000 | AUROC:0.9954
Test | 30/16 | Loss:0.5326 | MainLoss:0.5326 | SPLoss:2.8269 | CLSLoss:0.0000 | AUROC:0.9905

Epoch: [503 | 1000] LR: 0.002612
Train | 16/16 | Loss:0.5361 | MainLoss:0.4562 | Alpha:0.0283 | SPLoss:2.8274 | CLSLoss:0.0000 | AUROC:0.0000
Test | 128/16 | Loss:0.2227 | MainLoss:0.2227 | SPLoss:2.8292 | CLSLoss:0.0000 | AUROC:0.9956
Test | 30/16 | Loss:0.5378 | MainLoss:0.5378 | SPLoss:2.8292 | CLSLoss:0.0000 | AUROC:0.9904

Epoch: [504 | 1000] LR: 0.002606
Train | 16